In [ ]:
#水域は2つのクラス（湿地と水域）を検出するように作絵師しているが、湿地帯についてはまだ精度の検証がされていない。

from ultralytics import YOLO
import cv2
import csv
import numpy as np
from osgeo import gdal
import os
import shapefile
from shapely.geometry import Polygon
import ast
import geopandas as gpd

In [ ]:
#フォルダーの指定。一行目がGeotiff化した外邦図のあるフォルダ。2，3、4は先に作成しておく。それぞれcsv、ポリゴンファイル、
#先のポリゴンファイルをユニオンで統一したShapeファイルが入っている。最後のフォルダが完成物。
#最後のモデルは、住宅地用にファインチューニングしたもの。このプログラムは、Windowsのユーザー直下、ultralyticsフォルダ内で動かしている。
#現在のモデルの位置は、そのフォルダ内でファインチューニングを行った際に自然に作成されるモデルの位置に同じ。
folder_path = 'D:/Gaihozu/Japan/Tohoku/'
folder_path2 = 'D:/YOLO/testdata3/test'
folder_path3 = 'D:/YOLO/testdata3/shape'
folder_path4 = 'D:/YOLO/testdata3/union'
model = YOLO('./runs/segment/yolov8_segmentation_water400_2/weights/best.pt')

In [ ]:
#ポリゴンの頂点を、画像の位置から緯度経度に変換するためのプログラム。
def length_of_pixel(image_path):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # ラスターファイルを開く
    dataset = gdal.Open(image_path)

    # ラスターの座標変換
    transform = dataset.GetGeoTransform()

    # ラスターの左上と右下の座標を取得
    min_x = transform[0]
    max_y = transform[3]
    max_x = min_x + transform[1] * dataset.RasterXSize
    min_y = max_y + transform[5] * dataset.RasterYSize

    # 一画素ごとの長さを計算
    kx = (max_x - min_x) / width
    ky = (max_y - min_y) / height

    # データセットを閉じる
    dataset = None

    return kx, ky, min_x, max_y


In [ ]:
#モデルを作って、水域を検出するプログラム
def split_image_and_predict(image_path, output_dir, base_name, tile_size=640, overlap_size=10, overlap_threshold=0.25):
    # 画像の読み込み
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    kx, ky, min_lx, max_ly = length_of_pixel(image_path)

    # 出力ディレクトリが存在しない場合は作成
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    y = 0
    while y < height:
        x = 0
        while x < width:
            # タイルのサイズを計算
            tile_height = min(tile_size, height - y)
            tile_width = min(tile_size, width - x)

            # x_step と y_step の初期化
            y_step = tile_size - overlap_size
            x_step = tile_size - overlap_size

            # 最後のタイルでは重なりを避ける
            if y + tile_height >= height:
                y_step = tile_size

            if x + tile_width >= width:
                x_step = tile_size

            # タイル画像の抽出
            tile = image[y:y + tile_height, x:x + tile_width]

            # YOLOv8モデルで予測
            results = model(tile)

            # 予測結果を取得
            for result in results:
                if result.masks is not None:
                    # クラスIDとポリゴン情報を取得
                    for mask, cls in zip(result.masks.xy, result.boxes.cls):
                        polygon_points = []

                        # 各マスクの頂点座標を変換
                        for point in mask:
                            abs_x, abs_y = point[0], point[1]
                            X = x + abs_x
                            Y = y + abs_y
                            Lx = min_lx + (X * kx)
                            Ly = max_ly - (Y * ky)
                            polygon_points.append((Lx, Ly))

                        # クラスIDをint型に変換
                        cls = int(cls)

                        # クラスごとのCSVファイルパスを決定
                        class_csv = os.path.join(output_dir, f'class_{cls}_{base_name}.csv')

                        # CSVファイルが存在しない場合はヘッダーを追加して新規作成
                        if not os.path.exists(class_csv):
                            with open(class_csv, mode='w', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(['Polygon', 'cls'])

                        # クラスごとのCSVファイルにポリゴン座標とクラスIDを書き込み
                        with open(class_csv, mode='a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow([polygon_points, cls])

            # 次のタイルへ
            x += x_step
        y += y_step


In [ ]:
# フォルダ内のすべての .tif ファイルに対して検出処理するためのプログラム
for file_name in os.listdir(folder_path):
    if file_name.endswith('.tif'):
        # 拡張子を除いたファイル名（ベース名）を取得
        base_name = os.path.splitext(file_name)[0]

        
        # 最初に使用するデータ
        image_path = os.path.join(folder_path, file_name)
        output_dir = folder_path2

        split_image_and_predict(image_path, output_dir, base_name,tile_size=640, overlap_size=100, overlap_threshold=0.25)

In [ ]:
#ポリゴン化するためのプログラム。ここで、class_0が名前の先頭につくのが湿地帯、class_1がつくのが水域。
def parse_polygon_line(line):
    """ ポリゴンの頂点データを解析する関数 """
    try:
        # 頂点データ部分を取得
        vertices_str = line.strip().strip('"')
        
        # 文字列からリストに変換
        vertices = ast.literal_eval(vertices_str)
        
        return vertices
    except Exception as e:
        print(f"Error parsing line: {e}")
        return None


for file_name in os.listdir(folder_path2):
    if file_name.endswith('.csv'):
        # 拡張子を除いたファイル名（ベース名）を取得
        base_name = os.path.splitext(file_name)[0]
        input_file = os.path.join(folder_path2, file_name)
        output_shapefile = os.path.join(folder_path3, f'{base_name}.shp')  # `basename`ではなく`base_name`
        
        # SHPファイルの作成
        with shapefile.Writer(output_shapefile, shapeType=shapefile.POLYGON) as shp:
            shp.field("ID", "N")  # 属性テーブルにフィールドを追加（整数型）
            shp.field("cls", "F", decimal=2)  # clsフィールドを浮動小数点数型で追加（小数点以下2桁まで）
      
            with open(input_file, "r") as f:
                reader = csv.reader(f)
                next(reader)  # ヘッダー行をスキップ
                for idx, row in enumerate(reader):
                    line = row[0]  # 最初の列にポリゴンデータが含まれている
                    cls_value = float(row[1])  # 2列目にclsの値が小数点付きで含まれているので、floatに変換

                    # ポリゴンの頂点データを解析
                    vertices = parse_polygon_line(line)
                    if vertices:
                        # ポリゴンを作成
                        polygon = Polygon(vertices)
                
                        # ポリゴンをSHPファイルに書き込む
                        shp.record(idx, cls_value)  # IDフィールドとclsフィールドに値を記録
                        shp.poly([vertices])

        # .prjファイルの作成（WGS84座標系の定義）
        with open(output_shapefile.replace(".shp", ".prj"), "w") as prj:
            prj.write(
                'GEOGCS["WGS 84",'
                'DATUM["WGS_1984",'
                'SPHEROID["WGS 84",6378137,298.257223563]],'
                'PRIMEM["Greenwich",0],'
                'UNIT["degree",0.0174532925199433]]'
            )

print("SHPファイルが作成されました。")


In [ ]:
#検出を行うと、同じ個所を複数回検出する場合がある。その重なりを修正するために、Unionコマンドを使用した。
import os
import geopandas as gpd

# union_shp関数の定義
def union_shp(input_file, output_file):
    try:
        # ポリゴンベクターデータを読み込みます
        gdf = gpd.read_file(input_file)

        # 無効なジオメトリを修正（buffer(0) を使用して自己交差などを修正）
        gdf['geometry'] = gdf['geometry'].buffer(0)

        # 全てのポリゴンのユニオンを計算します (union_all() を使用)
        union = gdf.geometry.union_all()

        # ユニオン結果を新しいGeoDataFrameとして保存します
        union_gdf = gpd.GeoDataFrame(geometry=[union], crs=gdf.crs)

        # ユニオンを新しいシェープファイルとして保存します
        union_gdf.to_file(output_file)
        print(f"{output_file}が作成されました。")
    except Exception as e:
        print(f"{input_file}の処理中にエラーが発生しました: {e}")

# 入力フォルダ内のすべての.shpファイルにunion_shpを適用
for file_name in os.listdir(folder_path3):
    if file_name.endswith('.shp'):
        # ベース名を取得
        base_name = os.path.splitext(file_name)[0]
        input_file = os.path.join(folder_path3, file_name)
        output_file = os.path.join(folder_path4, f'union_{base_name}.shp')
        union_shp(input_file, output_file)
